In [1]:
# load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# tested agent creation and invocation with gpt-5-nano
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agent = create_agent(model="gpt-5-nano")
question = HumanMessage(content="What is the capital of India?")

response = agent.invoke({"messages": [question]}) 
print(response['messages'][1].content)

New Delhi. It is the capital city of India and houses the central government’s key institutions.


In [ ]:
# Lets create a tool for agent to use
from langchain.tools import Tool

def web_search(query: str) -> str:
    # This is a placeholder for a real web search function
    return f"Results for '{query}'"